In [2]:
using LinearAlgebra, Plots, Random, Distributions, NLsolve

First, let us create a NamedTuple with our parameters.

In [3]:
params = (; β = 0.72, # bargaining_power
            r = 0.012, # discount rate
            λ = 4, # arrival rate
            z = 0.4, # value of leisure
            p_star = 1., # productivity
            s_star = 0.1, # separation rate
            c = 0.213, # cost of vacancy
            σ = 0.0165, # standard deviation 
            γ = 0.004, # autoregressive coefficient
            n = 2001, # gridpoints
            lb = -1., # bounds for grid
            ub = 1., # bounds for grid
            mc = 1.355, # constant term in matching function
            me = -0.72, # exponent in matching function
            max_iter  = 1000 )

(β = 0.72, r = 0.012, λ = 4, z = 0.4, p_star = 1.0, s_star = 0.1, c = 0.213, σ = 0.0165, γ = 0.004, n = 2001, lb = -1.0, ub = 1.0, mc = 1.355, me = -0.72, max_iter = 1000)

In [4]:
function matching_function(grid, params)
    (; mc, me) = params
    return mc .* (grid .^ me)
end

matching_function (generic function with 1 method)

In [5]:
function generate_income_process(params)
    (; n, lb, ub) = params
    Δ = ub / n
    grid = LinRange(lb, ub, n)
    exp_grid = exp.(grid)
    transition_matrix = zeros(n, n)
    for i in 1:n
        if i == 1
            transition_matrix[i, i] = ( 1/2 ) * (1 .+ (grid[i] / (n*Δ)))
            transition_matrix[i, i+1] = ( 1/2 ) * (1 .- (grid[i] / (n*Δ)))
        elseif i == n
            transition_matrix[i, i-1] = ( 1/2 ) * (1 .+ (grid[i] / (n*Δ)))
            transition_matrix[i, i] = ( 1/2 ) * (1 .- (grid[i] / (n*Δ)))
        else
            transition_matrix[i, i-1] = ( 1/2 ) * (1 .+ (grid[i] / (n*Δ)))
            transition_matrix[i, i+1] = ( 1/2 ) * (1 .- (grid[i] / (n*Δ)))
        end
    end
    return exp_grid, transition_matrix
end

income_process, transition_matrix = generate_income_process(params)

([0.36787944117144233, 0.3682475046136629, 0.3686159363034188, 0.3689847366091417, 0.369353905899632, 0.3697234445440589, 0.3700933529119613, 0.37046363137324734, 0.37083428029819565, 0.37120530005745517  …  2.6939270528874992, 2.696622327353013, 2.6993202984410796, 2.7020209688496686, 2.7047243412794524, 2.7074304184338023, 2.7101392030187967, 2.7128506977432196, 2.715564905318567, 2.718281828459045], [0.0 1.0 … 0.0 0.0; 0.0004999999999999449 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0004999999999999449; 0.0 0.0 … 1.0 0.0])

In [21]:
function productivity_grid(params)
    (; z, p_star) = params
    grid = generate_income_process(params)
    p_grid = z .+ (grid[1] .* (p_star - z))
    return p_grid
end

function separation_grid(params)
    (; n, s_star) = params
    grid = generate_income_process(params)
    s_grid = grid[1] .* s_star
    return s_grid
end

function theta_grid(params)
    (; n) = params
    grid = generate_income_process(params)
    θ_grid = LinRange(0.1,3,n)
    return θ_grid
end

theta_grid (generic function with 1 method)

In [27]:
function iterate_prod(grid, params)
    (; β, r, λ, z, p_star, s_star, c, σ, γ, n, lb, ub, mc, me, max_iter) = params
    Δ = ub / n
    exp_grid, transition_matrix = generate_income_process(params)
    prod_grid = productivity_grid(params)
    iter = 0
    error = Inf
    new_grid = zeros(n)
    while iter < max_iter && error > 0.001
        iter += 1
        match_grid = matching_function(grid, params)
        for i in 1:n
            if i == 1
                result = nlsolve(theta -> - ((r+s_star+λ) / match_grid[i]) + (((1-β)*(prod_grid[i]-z)) / c ) - β*theta[1] + λ*((transition_matrix[i, i+1] / match_grid[i+1])), [grid[i]])
                new_grid[i] = result.zero[1]
            elseif i == n
                result = nlsolve(theta -> - ((r+s_star+λ) / match_grid[i]) + (((1-β)*(prod_grid[i]-z)) / c ) - β*theta[1] + λ*((transition_matrix[i, i-1] / match_grid[i-1])), [grid[i]])
                new_grid[i] = result.zero[1]
            else
                result = nlsolve(theta -> - ((r+s_star+λ) / match_grid[i]) + (((1-β)*(prod_grid[i]-z)) / c ) - β*theta[1] + λ*((transition_matrix[i, i-1] / match_grid[i-1]) + (transition_matrix[i, i+1] / match_grid[i+1])), [grid[i]])
                new_grid[i] = result.zero[1]
            end
        end
        error = norm(new_grid - grid)
        grid = copy(new_grid)
        if iter % 1 == 0
            println("Iteration: $iter, Error: $error")
        end
    end
    return grid
end

θ_grid = theta_grid(params)
thetas = iterate_prod(θ_grid, params)

Iteration: 1, Error: 22.24042821451652
Iteration: 2, Error: 1.645337822710512
Iteration: 3, Error: 0.13490677881814264
Iteration: 4, Error: 0.012028792790704025
Iteration: 5, Error: 0.0012228452433972544
Iteration: 6, Error: 0.00014429373642639175


2001-element Vector{Float64}:
 0.3504679344054823
 0.3508306262460641
 0.3511936493759864
 0.3515570985052328
 0.351920871184671
 0.3522850393677497
 0.35264958272735075
 0.3530144697176859
 0.3533797712593901
 0.35374541566147344
 ⋮
 2.7124282294989412
 2.7151998609556762
 2.717974312261973
 2.7207515685657864
 2.723531674825363
 2.726314585161308
 2.7291003336030744
 2.7318889231898624
 2.734680325969482

In [28]:
function iterate_sep(grid, params)
    (; β, r, λ, z, p_star, s_star, c, σ, γ, n, lb, ub, mc, me, max_iter) = params
    Δ = ub / n
    exp_grid, transition_matrix = generate_income_process(params)
    sep_grid = separation_grid(params)
    iter = 0
    error = Inf
    new_grid = zeros(n)
    while iter < max_iter && error > 0.001
        iter += 1
        match_grid = matching_function(grid, params)
        for i in 1:n
            if i == 1
                result = nlsolve(theta -> - ((r+sep_grid[i]+λ) / match_grid[i]) + (((1-β)*(p_star-z)) / c ) - β*theta[1] + λ*((transition_matrix[i, i+1] / match_grid[i+1])), [grid[i]])
                new_grid[i] = result.zero[1]
            elseif i == n
                result = nlsolve(theta -> - ((r+sep_grid[i]+λ) / match_grid[i]) + (((1-β)*(p_star-z)) / c ) - β*theta[1] + λ*((transition_matrix[i, i-1] / match_grid[i-1])), [grid[i]])
                new_grid[i] = result.zero[1]
            else
                result = nlsolve(theta -> - ((r+sep_grid[i]+λ) / match_grid[i]) + (((1-β)*(p_star-z)) / c ) - β*theta[1] + λ*((transition_matrix[i, i-1] / match_grid[i-1]) + (transition_matrix[i, i+1] / match_grid[i+1])), [grid[i]])
                new_grid[i] = result.zero[1]
            end
        end
        error = norm(new_grid - grid)
        grid = copy(new_grid)
        if iter % 1 == 0
            println("Iteration: $iter, Error: $error")
        end
    end
    return grid
end

θ_grid = theta_grid(params)
sep_thetas = iterate_sep(θ_grid, params)
    

Iteration: 1, Error: 54.000472622261384
Iteration: 2, Error: 7.700569011569239
Iteration: 3, Error: 1.4943263408370036
Iteration: 4, Error: 0.29482678774158727
Iteration: 5, Error: 0.0614452372134459
Iteration: 6, Error: 0.013214677460164956
Iteration: 7, Error: 0.002923860883392916
Iteration: 8, Error: 0.000711446442226081


2001-element Vector{Float64}:
 1.0437770409880436
 1.0437396761402775
 1.0437019834920682
 1.043664469796426
 1.0436265391864397
 1.0435891756585358
 1.0435511573978233
 1.0435135185145556
 1.043475639612994
 1.0434378362402184
 ⋮
 0.8411663056806334
 0.8410053785000571
 0.8407777146815644
 0.8405965475521828
 0.8403823477397983
 0.8401908473220112
 0.8399995931775783
 0.8397657141037753
 0.8396222563331162

In [38]:
function bargaining_grid(params)
    (; n) = params
    lower_bound = 0.64
    upper_bound = 0.80
    grid = LinRange(lower_bound, upper_bound, n)    
    return reverse(grid)
end

barg_grid = bargaining_grid(params)

2001-element LinRange{Float64, Int64}:
 0.8, 0.79992, 0.79984, 0.79976, 0.79968, …, 0.64024, 0.64016, 0.64008, 0.64

In [39]:
function iterate_prod_barg(grid, params)
    (; r, λ, z, p_star, s_star, c, σ, γ, n, lb, ub, mc, me, max_iter) = params
    Δ = ub / n
    exp_grid, transition_matrix = generate_income_process(params)
    prod_grid = productivity_grid(params)
    β = bargaining_grid(params)
    iter = 0
    error = Inf
    new_grid = zeros(n)
    while iter < max_iter && error > 0.001
        iter += 1
        match_grid = matching_function(grid, params)
        for i in 1:n
            if i == 1
                result = nlsolve(theta -> - ((r+s_star+λ) / match_grid[i]) + (((1-β[i])*(prod_grid[i]-z)) / c ) - β[i]*theta[1] + λ*((transition_matrix[i, i+1] / match_grid[i+1])), [grid[i]])
                new_grid[i] = result.zero[1]
            elseif i == n
                result = nlsolve(theta -> - ((r+s_star+λ) / match_grid[i]) + (((1-β[i])*(prod_grid[i]-z)) / c ) - β[i]*theta[1] + λ*((transition_matrix[i, i-1] / match_grid[i-1])), [grid[i]])
                new_grid[i] = result.zero[1]
            else
                result = nlsolve(theta -> - ((r+s_star+λ) / match_grid[i]) + (((1-β[i])*(prod_grid[i]-z)) / c ) - β[i]*theta[1] + λ*((transition_matrix[i, i-1] / match_grid[i-1]) + (transition_matrix[i, i+1] / match_grid[i+1])), [grid[i]])
                new_grid[i] = result.zero[1]
            end
        end
        error = norm(new_grid - grid)
        grid = copy(new_grid)
        if iter % 1 == 0
            println("Iteration: $iter, Error: $error")
        end
    end
    return grid
end

θ_grid = theta_grid(params)
thetas_barg = iterate_prod_barg(θ_grid, params)

Iteration: 1, Error: 20.095365965672737
Iteration: 2, Error: 1.5756105466694514
Iteration: 3, Error: 0.13157801754922868
Iteration: 4, Error: 0.01142946280340591
Iteration: 5, Error: 0.0010867025345807162
Iteration: 6, Error: 0.00012097652090862869


2001-element Vector{Float64}:
 0.22514867331977384
 0.22549511191916263
 0.22584201765365652
 0.2261894348516834
 0.2265373416182277
 0.22688574386916102
 0.2272346394659428
 0.2275840402300056
 0.22793393091317585
 0.22828432428018341
 ⋮
 3.904921692787293
 3.9102604083157284
 3.9156063117875815
 3.920959357162226
 3.9263195420176134
 3.9316869376197885
 3.937061459077796
 3.942443251550919
 3.9478321392164313

In [41]:
function iterate_sep_barg(grid, params)
    (; r, λ, z, p_star, s_star, c, σ, γ, n, lb, ub, mc, me, max_iter) = params
    Δ = ub / n
    exp_grid, transition_matrix = generate_income_process(params)
    sep_grid = separation_grid(params)
    β = bargaining_grid(params)
    iter = 0
    error = Inf
    new_grid = zeros(n)
    while iter < max_iter && error > 0.001
        iter += 1
        match_grid = matching_function(grid, params)
        for i in 1:n
            if i == 1
                result = nlsolve(theta -> - ((r+sep_grid[i]+λ) / match_grid[i]) + (((1-β[i])*(p_star-z)) / c ) - β[i]*theta[1] + λ*((transition_matrix[i, i+1] / match_grid[i+1])), [grid[i]])
                new_grid[i] = result.zero[1]
            elseif i == n
                result = nlsolve(theta -> - ((r+sep_grid[i]+λ) / match_grid[i]) + (((1-β[i])*(p_star-z)) / c ) - β[i]*theta[1] + λ*((transition_matrix[i, i-1] / match_grid[i-1])), [grid[i]])
                new_grid[i] = result.zero[1]
            else
                result = nlsolve(theta -> - ((r+sep_grid[i]+λ) / match_grid[i]) + (((1-β[i])*(p_star-z)) / c ) - β[i]*theta[1] + λ*((transition_matrix[i, i-1] / match_grid[i-1]) + (transition_matrix[i, i+1] / match_grid[i+1])), [grid[i]])
                new_grid[i] = result.zero[1]
            end
        end
        error = norm(new_grid - grid)
        grid = copy(new_grid)
        if iter % 1 == 0
            println("Iteration: $iter, Error: $error")
        end
    end
    return grid
end

θ_grid = theta_grid(params)
sep_thetas = iterate_sep_barg(θ_grid, params)

Iteration: 1, Error: 45.474316076776056
Iteration: 2, Error: 6.846475992927783
Iteration: 3, Error: 1.3201216451684383
Iteration: 4, Error: 0.26430013613346603
Iteration: 5, Error: 0.055789793551270284
Iteration: 6, Error: 0.012175779014866875
Iteration: 7, Error: 0.0027348862772769906
Iteration: 8, Error: 0.0006594973776237966


2001-element Vector{Float64}:
 0.6713666915893561
 0.6716778304360874
 0.6719889380856261
 0.6723001499223206
 0.672611201407783
 0.6729225294044114
 0.6732336434657564
 0.6735449124293169
 0.6738562003486475
 0.6741673679889543
 ⋮
 1.2079839606664111
 1.2081191258054373
 1.2082316062953533
 1.2083532277483562
 1.208469303179048
 1.208599629669544
 1.2087045705270776
 1.20883275541049
 1.2089486087360286